# Local Evaluation - Groundedness

After you have setup and configured the prompt flow, its time to evaluation its performance. Here we can use the prompt flow SDK to test different questions and see how the prompt flow performs using the evaluation prompt flows provided.

In [15]:
from promptflow import PFClient
pf_client = PFClient()

from dotenv import load_dotenv

from pathlib import Path
load_dotenv(Path("../local.env"))

True

In [16]:
# Add a question to test the base prompt flow.
question = "How do I wash the jacket I purchased?"
customerId = "4"
output = pf_client.test(
    flow="../contoso-intent", # Path to the flow directory
    inputs={ # Inputs to the flow
        "chat_history": [],
        "question": question,
        "customerId": customerId,
    },
)

output["answer"] = "".join(list(output["answer"]))

[2024-01-03 18:28:38,251][promptflow][WARNING] - Unknown input(s) of flow: {'customerId': '4'}


[2024-01-03 18:28:38,262][promptflow.contracts.tool][WARNING] - Failed to check if list[str] is a custom strong type: issubclass() arg 1 must be a class
[2024-01-03 18:28:38,273][promptflow._sdk.entities._connection][WARNING] - Please use connection.secrets[key] to access secrets.
[2024-01-03 18:28:38,273][promptflow._sdk.entities._connection][WARNING] - Please use connection.secrets[key] to access secrets.
[2024-01-03 18:28:38,288][promptflow.contracts.tool][WARNING] - Failed to check if list[str] is a custom strong type: issubclass() arg 1 must be a class


2024-01-03 18:28:38 -0600   11880 execution.flow     INFO     Start executing nodes in thread pool mode.
2024-01-03 18:28:38 -0600   11880 execution.flow     INFO     Start to run 3 nodes with concurrency level 16.
2024-01-03 18:28:38 -0600   11880 execution.flow     INFO     Executing node classify_intent_prompt. node run id: 87274ab2-a061-4eaf-8b81-6804cae16576_classify_intent_prompt_0
2024-01-03 18:28:38 -0600   11880 execution.flow     INFO     Node classify_intent_prompt completes.
2024-01-03 18:28:38 -0600   11880 execution.flow     INFO     Executing node classify_intent_llm. node run id: 87274ab2-a061-4eaf-8b81-6804cae16576_classify_intent_llm_0
2024-01-03 18:28:39 -0600   11880 execution.flow     INFO     Node classify_intent_llm completes.
2024-01-03 18:28:39 -0600   11880 execution.flow     INFO     Executing node run_chat_or_support. node run id: 87274ab2-a061-4eaf-8b81-6804cae16576_run_chat_or_support_0
2024-01-03 18:28:39 -0600   11880 execution.flow     INFO     [run_cha

In [17]:
output

{'answer': '{"answer":"Hi Emily! To wash the Summit Breeze Jacket you purchased, machine wash it on a gentle cycle with cold water and air dry. Avoid using bleach or fabric softeners. Happy hiking! \\ud83e\\uddfc\\ud83c\\udf3f\\n\\nReference: # Information about product item_number: 3","citations":[{"content":"# Information about product item_number: 17\\n\\nRainGuard Hiking Jacket, price $110,\\n\\n\\n\\nWear appropriate layers underneath the jacket based on the weather conditions.\\nAdjust the hood, cuffs, and hem to achieve a snug and comfortable fit.\\nUtilize the pockets for storing small items such as keys, wallet, or a mobile phone.\\nIf needed, open the ventilation zippers to regulate airflow and prevent overheating.\\nBe mindful of the jacket\'s limitations in extreme weather conditions.\\n\\nCare and Maintenance\\n   To ensure the longevity and performance of your RainGuard Hiking Jacket, please adhere to the following care instructions:\\n\\nClean the jacket as needed follow

Test the groundedness of the prompt flow with the answer from the above question.

In [18]:
test = pf_client.test(
    flow="intent_eval",
    inputs={
        "question": question,
        "prediction": str(output["intent_context"]),
        "groundtruth": "support",
    },
)

[2024-01-03 18:28:42,011][promptflow][WARNING] - Unknown input(s) of flow: {'prediction': 'intent: support'}


2024-01-03 18:28:42 -0600   11880 execution.flow     INFO     Start executing nodes in thread pool mode.
2024-01-03 18:28:42 -0600   11880 execution.flow     INFO     Start to run 3 nodes with concurrency level 16.
2024-01-03 18:28:42 -0600   11880 execution.flow     INFO     Executing node llm_call. node run id: 67e59c6b-1c5c-4e50-b9d9-b37261a3e8f9_llm_call_0
2024-01-03 18:28:42 -0600   11880 execution.flow     INFO     Node llm_call completes.
2024-01-03 18:28:42 -0600   11880 execution.flow     INFO     Executing node assert_value. node run id: 67e59c6b-1c5c-4e50-b9d9-b37261a3e8f9_assert_value_0
2024-01-03 18:28:42 -0600   11880 execution.flow     INFO     Node assert_value completes.
2024-01-03 18:28:42 -0600   11880 execution.flow     INFO     Executing node get_accuracy. node run id: 67e59c6b-1c5c-4e50-b9d9-b37261a3e8f9_get_accuracy_0
2024-01-03 18:28:42 -0600   11880 execution.flow     INFO     Node get_accuracy completes.


In [19]:
test

{'results': {'accuracy': 100.0}}

# AI Studio Azure batch run on an evaluation json dataset

Now in order to test these more thoroughly, we can use the Azure AI Studio to run batches of test data with the evaluation prompt flow on a larger dataset.

In [12]:
import json
# Import required libraries
from promptflow.azure import PFClient

# Import required libraries
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

In [13]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

Populate the `config.json` file with the subscription_id, resource_group, and workspace_name.

In [14]:
config_path = "../config.json"
pf_azure_client = PFClient.from_config(credential=credential, path=config_path)

Found the config file in: ..\config.json


Add the runtime from the AI Studio that will be used for the cloud batch runs.

In [9]:
# Update the runtime to the name of the runtime you created previously
runtime = "automatic"
# load flow
flow = "../contoso-intent"
# load data
data = "../data/intenttestdata.jsonl"

In [10]:
# get current time stamp for run name
import datetime
now = datetime.datetime.now()
timestamp = now.strftime("%Y_%m_%d_%H%M%S")
run_name = timestamp+"_intent_base_run"
print(run_name)

2024_01_03_180055_intent_base_run


Create a base run to use as the variant for the evaluation runs. 

_NOTE: If you get "'An existing connection was forcibly closed by the remote host'" run the cell again._

In [11]:
# create base run in Azure Ai Studio
base_run = pf_azure_client.run(
    flow=flow,
    data=data,
    column_mapping={
        # reference data
        "customerId": "${data.customerId}",
        "question": "${data.question}",
    },
    runtime=runtime,
    # create a display name as current datetime
    display_name=run_name,
    name=run_name
)
print(base_run)

[2024-01-03 18:00:56,857][promptflow][WARNING] - You're using automatic runtime, if it's first time you're using it, it may take a while to build runtime and request may fail with timeout error. Wait a while and resubmit same flow can successfully start the run.
Uploading contoso-intent (0.22 MBs): 100%|##########| 218902/218902 [00:01<00:00, 119108.99it/s]


[2024-01-03 18:01:01,896][promptflow.azure._restclient.flow_service_caller][INFO] - Start polling until session creation is completed...
[2024-01-03 18:01:09,029][promptflow.azure._restclient.flow_service_caller][INFO] - Session creation finished with status Succeeded.


RunOperationParameterError: Unsupported workspace kind: 'hub'

In [ ]:
pf_azure_client.stream(base_run)

In [ ]:
details = pf_azure_client.get_details(base_run)
details.head(10)

# Cloud Eval run on Json Data

In [ ]:
eval_flow = "intent_eval/"
run_name = timestamp+"intent_eval_run"
print(run_name)

eval_run_variant = pf_azure_client.run(
    flow=eval_flow,
    data=data,  # path to the data file
    run=base_run,  # use run as the variant
    column_mapping={
        # reference data
        "question": "${data.question}",
        "groundtruth": "${data.intent}",
        "prediction": "${run.intent_context}",
    },
    runtime=runtime,
    display_name=run_name,
    name=run_name
)

In [ ]:
pf_azure_client.stream(eval_run_variant)

In [ ]:
details = pf_azure_client.get_details(eval_run_variant)
details.head(10)

In [ ]:

metrics = pf_azure_client.get_metrics(eval_run_variant)
print(json.dumps(metrics, indent=4))

In [ ]:
pf_azure_client.visualize([base_run, eval_run_variant])